# Coding Project

You've covered a lot of material within this chapter; all the way from beginning to learn the basics of Python as a programming language, intermediate and advanced concepts, infinite iteration and complex string operations. Just to quickly recap:

1. __Fundamentals of Python__: Strings, for loops, lists, dictionaries, if statements
2. __Intermediate concepts__: Sets, decorators, IO, Exceptions, `with`, `lambda`
3. __Itertools__: Generators, `map`, `zip`, iteration, infinite series
4. __Regular expressions__: String manipulation, pattern matching

## Developing a game of Hearts

For this project, you're going to be writing a game of Hearts from scratch. The rules of the game are as follows:

1. There are 2-4 players.
2. Players are initially dealt 13 cards (with 4 players) which they can see.
3. The aim is to minimize the number of points they gain over the game. Points are given for each **hand** the player wins.
4. *Heart*-suited cards are worth 1 point, the Black Widow (Queen of Spades) is worth 13 points. 
5. Each hand happens when each player in turn plays one of their cards in the middle. The player with the 2 of Clubs starts, going clockwise. The suit of the first players' card must be followed by all subsequent players, *unless* they do not have a card of that suit. Then they can play whatever card they want.
6. The player who plays the highest-value card (Ace counts as high), wins the hand. 
7. There is no *trump* suit that beats other suits.
8. The game is played until everyone runs out of cards.

## Tasks

Your general aims are several-fold:

1. Create a deck of cards
2. Implement a class `Game` which has the state of the board, and a record of all the played cards.
3. Implement a class `Player` that receives a random selection of cards and can make informed decisions, based on the state of the game, which card to play in the next round. This player will act as an AI.
4. Run the simulation and calculate how many points each AI player gets, to decide who wins.
5. Repeat the simulation multiple times to see if a pattern emerges.

In [1]:
import itertools as it
import random

In [74]:
numbers = [str(a) for a in [2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K", "A"]]
suits = ["H", "S", "D", "C"]
corder = dict(zip(numbers, range(len(numbers))))

In [58]:
random.shuffle(list(it.product(numbers,suits)))

In [309]:
def argmax(R):
    amax = lambda i: R[i]
    return max(range(len(R)), key=amax)

def argmin(R):
    amin = lambda i: R[i]
    return min(range(len(R)), key=amin)

def sample(S):
    return S[random.randint(0, len(S)-1)]


class Player(object):
    """
    This player acts an AI who plays Hearts.
    """
    def __init__(self, player_id, cards, game_instance):
        # the cards are given directly to the player.
        self.id = player_id
        self.hand = list(cards)
        self.g = game_instance
        
    def _n_hearts_in_board(board):
        return len([c for c in board if "H" in c])
    
    def _widow_in_board(board):
        return "Q_S" in board
    
    def _rank_order_board(board):
        return [corder[c.split("_")[0]] for c in board]
    
    def _max_card(rank, cardset):
        # returns the highest value card out of the ranked cards
        return cardset[argmax(rank)]
    
    def _min_card(rank, cardset):
        return cardset[argmin(rank)]
        
    def _cards_of_suit(self, suit):
        return [c for c in self.hand if c.endswith(suit)]
    
    def _rank_order_cards(self):
        # returns the rank of the cards in the hand
        return [corder[c.split("_")[0]] for c in self.hand]
    
    def _rank_order_suit_cards(self, suit="H"):
        return [corder[c.split("_")[0]] for c in self.hand if suit in c]
    
    def _rank_order_nonheart_cards(self):
        return [corder[c.split("_")[0]] for c in self.hand if "H" not in c]
    
    def play(self, n_hand, board=[], start=False):
        # if its the start, if you have the 2C, play it
        if start and (g.start_card in self.hand):
            self.hand.remove(g.start_card)
            return g.start_card
        else:
            """
            Rules for playing: priority:
            CHECK THE BOARD for first card played: sets the suit
            IF: No card played THEN:
                IF: Early in game, then play HIGH card (A, K, Q) of NON-heart suit
                ELIF: Late game, then play LOW card of ANY suit
            ELIF: Other cards played THEN
                IF Has a suit THEN
                    IF Hearts are present in board OR black widow THEN:
                        play lowest card of that suit
                    ELIF Hearts not present in board AND black widow not present THEN:
                        play any card of that suit
                ELIF: Doesn't have a suit, AND suit is not heart, THEN play highest heart or black widow
                ELIF: Doesn't have a suit, AND suit is heart, THEN play highest card ANY suit
            """
            if len(board) == 0:
                # beginning player - check hand and number of hands already completed
                if n_hand < 3:
                    # find highest non-heart card
                    NH = Player._max_card(self._rank_order_nonheart_cards(), self.hand)
                    # play it
                    self.hand.remove(NH)
                    return NH
                else:
                    # find lowest ANY card
                    LA = Player._min_card(self._rank_order_cards(), self.hand)
                    self.hand.remove(LA)
                    return LA
            else:
                # check which suit we are in from the first card in board
                this_suit = board[0].split("_")[1]
                # do we have any of that suit?
                cards_of_suit = self._cards_of_suit(this_suit)
                # rank the board
                board_rank = Player._rank_order_board(board)
                # count the number of hearts on board
                n_hearts = Player._n_hearts_in_board(board)
                # is widow present
                with_widow = Player._widow_in_board(board)
                
                # if we have this suit in our hand, we must play one
                if len(cards_of_suit) > 0:
                    # rank our cards
                    card_suit_rank = self._rank_order_suit_cards(this_suit)
                    if (n_hearts > 0) or with_widow:
                        # play the minimum card of that suit
                        MC = Player._min_card(card_suit_rank, cards_of_suit)
                        self.hand.remove(MC)
                        return MC
                    elif (n_hearts == 0) and (not with_widow):
                        # randomly sample a card
                        MC = sample(cards_of_suit)
                        self.hand.remove(MC)
                        return MC
                    else:
                        raise ValueError("Shouldn't be able to get here!")
                else:
                    # we don't have the suit in our hand, not leading
                    if this_suit == "H":
                        # we don't have any cards, play largest card any other suit
                        MC = Player._max_card(self._rank_order_cards(), self.hand)
                        self.hand.remove(MC)
                        return MC
                    else:
                        if "Q_S" in self.hand:
                            self.hand.remove("Q_S")
                            return "Q_S"
                        else:
                            # play our largest heart
                            card_hearts = self._rank_order_suit_cards("H")
                            if len(card_hearts) > 0:
                                MC = Player._max_card(card_hearts, self._cards_of_suit("H"))
                                self.hand.remove(MC)
                                return MC
                            else:
                                # play largest any card
                                MC = Player._max_card(self._rank_order_cards(), self.hand)
                                self.hand.remove(MC)
                                return MC
    

class Game(object):
    """
    The main game object.
    """
    def __init__(self, n_players):
        
        self.start_card = "2_C"
        self.deck = map(lambda x: "_".join(x), it.product(numbers, suits))
        # shuffle the deck
        shuf_d = self.shuffle(self.deck)
        shuffled_deck = self.cut(shuf_d, 26)
        # initialise the players given N
        hands = self.deal(shuffled_deck, n_players, 52//n_players)
        # create players and give them their cards
        self.players = [Player(i, h, self) for i,h in enumerate(hands)]
        self.total_hands = len(self.players[0].hand)
        # set up a discard pile
        self.discards = []
        
    def shuffle(self, deck):
        d = list(deck)
        random.shuffle(d)
        return iter(tuple(d))
    
    def cut(self, deck, n):
        # cut the deck
        deck1, deck2 = it.tee(deck, 2)
        top = it.islice(deck1, n)
        bottom = it.islice(deck2, n ,None)
        return it.chain(bottom, top)
    
    def deal(self, deck, num_hands, hand_size):
        iters = [iter(deck)] * hand_size
        return tuple(zip(*(tuple(it.islice(itr, num_hands)) for itr in iters)))
    
    def play_hand(self, n, player_loop, begin=False):
        # begins a 'hand' and goes around the players, playing cards. If begin is true, player 1 plays their 2C
        hand = []
        card = player_loop[0].play(board=hand, n_hand=n, start=begin)
        self.discards.append(card)
        hand.append(card)
        # everyone else plays a card
        for i in range(1,len(player_loop)):
            c = player_loop[i].play(n_hand=n, board=hand, start=False)
            self.discards.append(c)
            hand.append(c)
        return hand
    
    def find_highest_suited_card(self, hand):
        the_suit = hand[0].split("_")[1]
        ranked = Player._rank_order_board(hand)
        return argmax([r*(the_suit in hand[i]) for i,r in enumerate(ranked)])
    
    def start(self):
        """
        Assumes all players are ready to go. This function begins the game after initialization.
        """
        self.hand_sets = []
        # find the player with the 2C
        i, player = [(p.id, p) for p in self.players if self.start_card in p.hand][0]
        player_loop = list(it.islice(it.dropwhile(lambda p: p.id != i, it.cycle(self.players)), 0, len(self.players)))
        print(i,player,player_loop)
        # play the first hand
        h = self.play_hand(1, player_loop, begin=True)
        # append
        self.hand_sets.append({"hand": h, "win_card": h[self.find_highest_suited_card(h)], "win_player":player_loop[self.find_highest_suited_card(h)].id})
        # set j
        j = 2
        while j < self.total_hands+1:
            # choose next start player based on highest win card in hand
            # create new player loop
            i, player = player_loop[self.find_highest_suited_card(h)].id, player_loop[self.find_highest_suited_card(h)]
            player_loop = list(it.islice(it.dropwhile(lambda p: p.id != i, it.cycle(self.players)), 0, len(self.players)))
            h = self.play_hand(j, player_loop, begin=False)
            # append
            self.hand_sets.append({"hand": h, "win_card": h[self.find_highest_suited_card(h)], "win_player":player_loop[self.find_highest_suited_card(h)].id})
            j += 1
            

In [312]:
g = Game(n_players=4)
g.start()
g.hand_sets

2 <__main__.Player object at 0x7fc2841ac5c0> [<__main__.Player object at 0x7fc2841ac5c0>, <__main__.Player object at 0x7fc2841ac4a8>, <__main__.Player object at 0x7fc2841ac470>, <__main__.Player object at 0x7fc2841ac128>]


[{'hand': ['2_C', '3_C', 'A_C', 'K_C'], 'win_card': 'A_C', 'win_player': 0},
 {'hand': ['K_D', '3_D', '9_D', '10_D'], 'win_card': 'K_D', 'win_player': 0},
 {'hand': ['3_H', '5_H', '7_H', '2_H'], 'win_card': '7_H', 'win_player': 2},
 {'hand': ['5_C', '4_C', '9_C', '6_C'], 'win_card': '9_C', 'win_player': 0},
 {'hand': ['4_H', '6_H', '8_H', '9_H'], 'win_card': '9_H', 'win_player': 3},
 {'hand': ['3_S', '8_S', '6_S', 'J_S'], 'win_card': 'J_S', 'win_player': 2},
 {'hand': ['8_C', '10_C', '7_C', 'Q_H'], 'win_card': '10_C', 'win_player': 3},
 {'hand': ['4_S', '7_S', '2_S', '10_S'], 'win_card': '10_S', 'win_player': 2},
 {'hand': ['9_S', '5_S', 'A_H', 'A_S'], 'win_card': 'A_S', 'win_player': 1},
 {'hand': ['2_D', 'J_D', '8_D', '5_D'], 'win_card': 'J_D', 'win_player': 2},
 {'hand': ['J_C', 'Q_C', 'J_H', '7_D'], 'win_card': 'Q_C', 'win_player': 3},
 {'hand': ['K_H', '10_H', '6_D', 'K_S'], 'win_card': 'K_H', 'win_player': 3},
 {'hand': ['A_D', 'Q_D', '4_D', 'Q_S'], 'win_card': 'A_D', 'win_player

In [313]:
len(g.discards)

52

In [316]:
g.players[0]